<div dir="rtl" style="direction: rtl; text-align: right;">
<h1>M2.4 - معالجة سجلات المناخ الطويلة بالتوازي</h1>
<p>جزء من: <a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science" dir="ltr">Computational Climate Science</a></p>
<hr>
<p><strong>المحتويات:</strong></p>
<ul>
<li>قيود الموارد في الحوسبة</li>
<li>المعالجة المتزامنة لبيانات المناخ الكبيرة</li>
<li>حساب PET باستخدام معادلة Hargreaves</li>
<li>تطبيق الدوال على الكتل المستقلة</li>
<li>قياس الأداء باستخدام timeit</li>
<li>الخلاصة</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>نظرة عامة</h2>
<p>
في الدرس السابق استخدمنا نموذج الدلو البسيط لقياس الفرق بين إمداد المياه (الهطول)
وفقدان المياه (التبخر-النتح المحتمل PET). يمكن التعبير عن نسبة التعويض كما يلي:
</p>
$$
\text{Percentage replenished} \approx 100 \times \frac{\text{Precipitation}}{\text{PET}}
$$
<p>
في هذا الدرس سنحسب PET باستخدام معادلة Hargreaves انطلاقًا من بيانات الحرارة،
وسنتعلم كيف يمكن معالجة البيانات المناخية الكبيرة باستخدام المعالجة المتزامنة.
</p>
</div>

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>قيود الموارد في الحوسبة</h2>
<p>
هناك ثلاثة أنواع رئيسية من القيود (عنق الزجاجة) في الحوسبة:
</p>
<ol>
<li>سرعة القراءة والكتابة من نظام الملفات (I/O-bound)</li>
<li>الذاكرة (Memory-bound)</li>
<li>سرعة وحدة المعالجة المركزية CPU (CPU-bound)</li>
</ol>
<p>
فهم نوع القيد يساعدنا على اختيار أفضل استراتيجية لتحسين الأداء.
</p>
</div>

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>المعالجة المتزامنة لبيانات المناخ</h2>
<p>
عند استخدام <code dir="ltr">xarray.open_mfdataset()</code>، يتم تمثيل البيانات
دون تحميلها مباشرة إلى الذاكرة (التقييم الكسول Lazy Evaluation).
</p>
<p>
يتم تقسيم البيانات إلى <strong>كتل (Chunks)</strong> يمكن معالجتها بشكل مستقل.
</p>
</div>

In [ ]:
import xarray as xr

ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4')
ds

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>حساب PET باستخدام معادلة Hargreaves</h2>
<p>
معادلة Hargreaves:
</p>
$$
\text{PET} = 0.0023 \times R_A \times \sqrt{T_{max} - T_{min}} \times (T + 17.8)
$$
<p>
حيث يمثل $R_A$ الإشعاع الشمسي أعلى الغلاف الجوي.
</p>
</div>

In [ ]:
import numpy as np

def potential_et(dataset):
    return 0.0023 * dataset['toa_radiation'] * \
           np.sqrt(dataset['T2MMAX'] - dataset['T2MMIN']) * \
           (dataset['T2MMEAN'] + 17.8)

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>تطبيق الدوال على الكتل المستقلة</h2>
<p>
يمكننا استخدام <code dir="ltr">xr.map_blocks()</code> لتطبيق دالة
على كل كتلة بشكل مستقل، مما يسمح باستخدام المعالجة المتزامنة.
</p>
</div>

In [ ]:
result = xr.map_blocks(potential_et, ds)
pet = result.compute()
pet

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>قياس الأداء باستخدام timeit</h2>
<p>
يمكن استخدام الأمر السحري <code dir="ltr">%%timeit</code> في Jupyter
لقياس الزمن الفعلي (Wall Time) لتنفيذ مهمة معينة.
</p>
</div>

<div dir="rtl" style="direction: rtl; text-align: right;">
<h2>الخلاصة</h2>
<ul>
<li>تستخدم xarray التقييم الكسول ولا تُحمِّل البيانات فورًا.</li>
<li>يمكن تقسيم البيانات إلى كتل لمعالجتها بالتوازي.</li>
<li>المعالجة المتزامنة مفيدة عندما يكون الحساب CPU-bound.</li>
<li>يجب اختيار حجم الكتل بعناية لتجنب الحمل الزائد.</li>
</ul>
</div>